In [1]:
import queueing_tool as qt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Define times
- higher rate for fast arrival time lambdas
- lower rate for slow arrival time lambdas

In [2]:
slow_rate = lambda t: 1.5
slow_arr = lambda t: qt.poisson_random_measure(t, slow_rate, 0.5)

fast_rate = lambda t: 0.25
fast_arr = lambda t: qt.poisson_random_measure(t, fast_rate, 0.5)

ser_t = lambda t: t + 0.2

### Define two Agents
- first with slow arr time
- second with fast arr time: 10 folds faster

In [3]:
def ag_slow(label='Slow'):
    return qt.Agent(
        agent_id = ('slow'),
        arrival_f = slow_arr,
        label= label
)

def ag_fast(label='Fast'):
    return qt.Agent(
        agent_id = ('fast'),
        arrival_f = fast_arr,
        label = label
)

### Setup the network

In [4]:
q_classes = { 1: qt.QueueServer, 2: qt.QueueServer }
q_args = {
    1: {
        'num_server': 1,
        'service_f': ser_t,
        'AgentFactory': ag_slow
    },
    2: {
        'num_server': 1,
        'service_f': ser_t,
        'AgentFactory': ag_fast
    }
}

# one starting point, two queues of Agents going into 3 and exit
adj_list = {0: [1, 2], 1: {3}, 2: {3}}
edge_list = {0: {1: 1}, 0: {2: 2}} 

### Instantiate graph and network

In [5]:
g = qt.adjacency2graph(
    adjacency=adj_list,
    edge_type=edge_list
)

qn = qt.QueueNetwork(
    g=g,
    q_classes=q_classes,
    q_args=q_args
)

### Simulate

In [6]:
qn.start_collecting_data()
qn.initialize(edge_type=[1, 2])
qn.simulate(n=100)  # simulate 100 events
data_out = qn.get_agent_data(return_header=True)

In [7]:
data_out

({'slow': array([[0.23799567, 0.23799567, 0.43799567, 0.        , 1.        ,
          2.        ],
         [0.40357363, 0.40357363, 0.60357363, 0.        , 1.        ,
          3.        ],
         [0.43799567, 0.        , 0.        , 0.        , 0.        ,
          4.        ],
         [0.58679616, 0.58679616, 0.78679616, 0.        , 1.        ,
          0.        ],
         [0.60357363, 0.        , 0.        , 0.        , 0.        ,
          4.        ],
         [0.78629014, 0.78629014, 0.        , 0.        , 1.        ,
          2.        ],
         [0.78679616, 0.        , 0.        , 1.        , 2.        ,
          2.        ],
         [0.91540315, 1.18629014, 1.38629014, 2.        , 3.        ,
          2.        ],
         [0.98629014, 0.        , 0.        , 0.        , 0.        ,
          4.        ],
         [1.18629014, 0.        , 0.        , 0.        , 0.        ,
          4.        ],
         [1.38629014, 0.        , 0.        , 0.        , 0.  

### Go through the data and select fast and slow agents

In [8]:
slow = list(map(lambda x: list(x), data_out[0]['slow']))
fast = list(map(lambda x: list(x), data_out[0]['fast']))

columns = (data_out[1]).split(',')
columns

['arrival', 'service', 'departure', 'num_queued', 'num_total', 'q_id']

### Convert to dataframes...

In [9]:
slow_df = pd.DataFrame(slow, columns=columns)
fast_df = pd.DataFrame(fast, columns=columns)

fast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   arrival     47 non-null     float64
 1   service     47 non-null     float64
 2   departure   47 non-null     float64
 3   num_queued  47 non-null     float64
 4   num_total   47 non-null     float64
 5   q_id        47 non-null     float64
dtypes: float64(6)
memory usage: 2.3 KB


In [10]:
slow_df['request'] = 'slow'
fast_df['request'] = 'fast'

### ...merge into one and add one column with the time spent in the network. Set to Nan if negative

 Time may be negative for not enough time for this agent to receive service   
 Remove NaN times

In [11]:
all_df = pd.concat([slow_df, fast_df])

all_df['times'] = all_df['departure'].sub(all_df['arrival'])

all_df['times'] = all_df['times'].apply(lambda x: x if x > 0 else pd.NA)
all_df.dropna()

,arrival,service,departure,num_queued,num_total,q_id,request,times
0,0.237996,0.237996,0.437996,0.0,1.0,2.0,slow,0.2
1,0.403574,0.403574,0.603574,0.0,1.0,3.0,slow,0.2
3,0.586796,0.586796,0.786796,0.0,1.0,0.0,slow,0.2
7,0.915403,1.186290,1.386290,2.0,3.0,2.0,slow,0.470887
11,1.807765,1.807765,2.007765,0.0,1.0,2.0,slow,0.2
13,2.315459,2.315459,2.515459,0.0,1.0,3.0,slow,0.2
14,2.332652,2.332652,2.532652,0.0,1.0,2.0,slow,0.2
17,2.901703,2.901703,3.101703,0.0,1.0,0.0,slow,0.2
18,3.101703,3.101703,3.301703,0.0,1.0,2.0,slow,0.2
20,3.476046,3.507874,3.707874,1.0,2.0,3.0,slow,0.231828


### Store the dataframe into excel file

In [12]:
all_df.to_excel('one_node_md_one.xlsx')